## TASK 4: Transfer learning from essentia models


The fourth task strategy is to transfer learning from a model that has already been transferred. For this purpose, a model trained to recognise voice/instrumental music audios proposed on Essentia TensorFlow is used as a starting point. This model is already been transferred from the VGGish model trained in Audioset. Therefore the layers trained by Audioset and an embedding layer trained in Essentia TensorFlow are used, and only the fully connected classifier is trained on top of all these layers.

In [1]:
! pip install -U pip
! pip install -U torch==1.5.1
! pip install -U torchaudio==0.5.1
! pip install -U torchvision==0.6.1
! pip install -U matplotlib==3.2.1
! pip install -U clearml>=0.16.1
! pip install -U pandas==1.0.4
! pip install -U numpy==1.18.4
! pip install -U tensorboard==2.2.1
!pip install git+https://github.com/mir-dataset-loaders/mirdata.git@Pedro/good_sounds
!pip install essentia
!pip install essentia-tensorflow
!wget https://essentia.upf.edu/models/classifiers/voice_instrumental/voice_instrumental-vggish-audioset-1.pb

     |████████████████████████████████| 1.5MB 11.2MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 753.2 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.9.0+cu101 requires torch==1.8.0, but you have torch 1.5.1 which is incompatible.
torchtext 0.9.0 requires torch==1.8.0, but you have torch 1.5.1 which is incompatible.
     |████████████████████████████████| 3.2 MB 11.3 MB/s 
     |████████████████████████████████| 6.6 MB 9.2 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:


     |████████████████████████████████| 10.1 MB 11.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.0.4 which is incompatible.


     |████████████████████████████████| 20.2 MB 41.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.9.0 requires torch==1.8.0, but you have torch 1.5.1 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.18.4 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.0.4 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 3.0 MB 11.1 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.18.4 which is incompatible.
tensorflow 2.4.1 requires tensorboard~=2.4, but you have tensorboard 2.2.1 which is incompatible.
  Cloning https://github.com/mir-dataset-loaders/mirdata.git (to revision Pedro/good_sounds) to /tmp/pip-req-build-6btq8yk4
  Running command git clone -q https://github.com/mir-dataset-loaders/mirdata.git /tmp/pip-req-build-6btq8yk4
  Running command git checkout -b Pedro/good_sounds --track origin/Pedro/good_sounds
  Switched to a new branch 'Pedro/good_sounds'
  Branch 'Pedro/good_sounds' set up to tr

In [4]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
if torch.cuda.is_available:
  print('GPU available')
else:
  print('Please set GPU via Edit -> Notebook Settings.')

NameError: ignored

In [5]:
import PIL
import io

import pandas as pd
import numpy as np
from pathlib2 import Path
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import torchaudio
from torchvision.transforms import ToTensor
from torchvision import models

%matplotlib inline

In [6]:

configuration_dict = {'number_of_epochs': 20, 'batch_size': 8, 'dropout': 0.3, 'base_lr': 0.005, 
                      'number_of_mel_filters': 64, 'resample_freq': 22050}


In [7]:
import mirdata

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

g = mirdata.initialize("good_sounds", data_home="drive/MyDrive/good_sounds")

Mounted at /content/drive


In [8]:
import os
import pickle

def emb_path(audio_path):
    # Embeddings path
    pre, ext = os.path.splitext(audio_path)
    return pre + '.pickle'

def emb_path_vgg(audio_path):
    # Embeddings path
    pre, ext = os.path.splitext(audio_path)
    dir, file = os.path.split(pre)
    return os.path.join(dir, 'vgg#' + file + '.pickle')

def emb_path_pen(audio_path):
    # Embeddings path
    pre, ext = os.path.splitext(audio_path)
    dir, file = os.path.split(pre)
    return os.path.join(dir, 'pen#' + file + '.pickle')

def store(data, filename):
    # Store data (serialize)
    with open(filename, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)


def load(filename):
    # Load data (deserialize)
    with open(filename, 'rb') as handle:
        unserialized_data = pickle.load(handle)
    return unserialized_data

### GENERATE INPUT

In [10]:
import os
from essentia.standard import MonoLoader, TensorflowPredictVGGish

def get_data(dataset, klasses):
    # import pdb; pdb.set_trace()
    sound_selected = []
    track_ids, tracks = [], []
    if klasses == "all":
        for k, t in dataset.load_tracks().items():
            if t.get_sound_info['klass'] and t.get_sound_info['id'] not in sound_selected:
                track_ids.append(k)
                tracks.append(t)
                sound_selected.append(t.get_sound_info['id'])
    else:
        for klass in klasses:
          for k, t in dataset.load_tracks().items():
              if t.get_sound_info['instrument'] == klass and t.get_sound_info['klass'] and t.get_sound_info['id'] not in sound_selected:
                  track_ids.append(k)
                  tracks.append(t)
                  sound_selected.append(t.get_sound_info['id'])
    return track_ids, tracks

count = 0
sizes = []

keys,tracks = get_data(g, ['violin', 'cello', 'bass'])
for k, t in zip(keys,tracks):
    if not os.path.exists(emb_path_vgg(t.audio_path)) or not os.path.exists(emb_path_pen(t.audio_path)):
        print(count)
        sr = 16000
        aud = MonoLoader(filename=t.audio_path, sampleRate=sr)()
        sizes.append(len(aud))
        audio = aud.copy()
        audio.resize(48000, refcheck=False)
        print(audio.shape, min(audio), max(audio))
        # Retrieve the output of the penultimate layer
        penultimate_layer = TensorflowPredictVGGish(graphFilename='/content/voice_instrumental-vggish-audioset-1.pb', output='model/fully_connected/BiasAdd')(audio)
        store(penultimate_layer, emb_path_pen(t.audio_path))
        # Retrieve weights of the last layer of VGGish
        vgg_embed = TensorflowPredictVGGish(graphFilename='/content/voice_instrumental-vggish-audioset-1.pb', output='model/vggish/fc2/BiasAdd')(audio)
        store(vgg_embed, emb_path_vgg(t.audio_path))

    count += 1


KeyboardInterrupt: ignored

In [ ]:
min(sizes), max(sizes)

(7693, 352179)

### DATASET LOADER

In [11]:
import sklearn 
import random
from essentia.standard import MonoLoader, TensorflowPredictMusiCNN

def get_data(dataset, klasses):
    # import pdb; pdb.set_trace()
    sound_selected = []
    track_ids, tracks = [], []
    if klasses == "all":
        for k, t in dataset.load_tracks().items():
            if t.get_sound_info['klass'] and t.get_sound_info['id'] not in sound_selected:
                track_ids.append(k)
                tracks.append(t)
                sound_selected.append(t.get_sound_info['id'])
    else:
        for klass in klasses:
          for k, t in dataset.load_tracks().items():
              if t.get_sound_info['instrument'] == klass and t.get_sound_info['klass'] and t.get_sound_info['id'] not in sound_selected:
                  track_ids.append(k)
                  tracks.append(t)
                  sound_selected.append(t.get_sound_info['id'])
    return track_ids, tracks


class good_soundsDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        mirdataset,
        seq_duration=0.5,
        random_start=True,
        resample=8000,
        subset=0,
        train_split=0.8,
        test_split=0.2,
        random_seed=32,
        klasses=["violin"]
    ):
        """
        """
        self.klasses = klasses # a 'klass' or 'all' or 'family'
        self.seq_duration = seq_duration
        self.dataset = mirdataset
        track_ids, tracks = get_data(self.dataset, self.klasses)
        self.track_ids = track_ids
        self.tracks = tracks
        self.resample = resample
        self.set = subset
        self.random_start = random_start

        #### build a list with labels
        self.labels = {label: i for i,label in enumerate(['good', 'bad'])}
        full_labels = [x.get_sound_info['klass'] for x in tracks]
        #### build the three subsets: train, validation, test using train_test_split, a stratified split with the labels
        self.trackids_train, self.trackids_test = sklearn.model_selection.train_test_split(self.track_ids, train_size=1-test_split, random_state=random_seed, stratify=full_labels)
        train_labels = [l for l,i in zip(full_labels, self.track_ids) if i in self.trackids_train]
        self.trackids_train, self.trackids_valid = sklearn.model_selection.train_test_split(self.trackids_train, train_size=train_split, random_state=random_seed, stratify=train_labels)


    def __getitem__(self, index):

        #### get the file with index in the corresponding subset
        if self.set==0:
            track_id = self.trackids_train[index]
        elif self.set==1:
            track_id = self.trackids_valid[index]
        elif self.set==2:
            track_id = self.trackids_test[index]
        track = self.dataset.track(track_id)

        
        embeddings = load(emb_path_pen(track.audio_path))
        embeddings = torch.tensor(embeddings)
        embeddings = torch.reshape(embeddings, (300,))
        # print(embeddings.shape)
        audio_signal = np.array([])

        return audio_signal, 16000, embeddings, self.labels['good' if track.get_sound_info['klass'].startswith('good') else 'bad']

    def __len__(self):
        if self.set==0:
            return len(self.trackids_train)
        elif self.set==1:
            return len(self.trackids_valid)
        else:
            return len(self.trackids_test)

random_seed=0

train_dataset = good_soundsDataset(mirdataset=g, subset=0, random_seed=random_seed)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64,num_workers=4,pin_memory=True)
valid_dataset = good_soundsDataset(mirdataset=g, subset=1, random_seed=random_seed)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=64,num_workers=4,pin_memory=True)
test_dataset = good_soundsDataset(mirdataset=g, subset=2, random_seed=random_seed)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64,num_workers=4,pin_memory=True)

classes = ('good', 'bad')
sounds, sample_rate, inputs, labels = train_dataset[0]
print(type(sounds), type(sample_rate), type(inputs), type(labels))

<class 'numpy.ndarray'> <class 'int'> <class 'torch.Tensor'> <class 'int'>


In [26]:
string = {
    "v" : {
        "k": ["violin"]
    },
    "c" : {
        "k": ["cello"]
    },
    "b" : {
        "k": ["bass"]
    },
    "all" : {
        "k": ["violin", "cello", "bass"]
    }
}
for k in string.keys():
    string[k]['train_dataset'] = good_soundsDataset(mirdataset=g, subset=0, random_seed=random_seed, klasses=string[k]['k'])
    string[k]['train_loader'] = torch.utils.data.DataLoader(string[k]['train_dataset'],batch_size=64,num_workers=4,pin_memory=True)
    string[k]['test_dataset'] = good_soundsDataset(mirdataset=g, subset=2, random_seed=random_seed, klasses=string[k]['k'])
    string[k]['test_loader'] = torch.utils.data.DataLoader(string[k]['test_dataset'],batch_size=64,num_workers=4,pin_memory=True)

In [ ]:
inputs.size()

torch.Size([300])

### FC LAYERS

In [13]:
import torch
import numpy as np

class myVGGish(torch.nn.Module):
    def __init__(self):
          super(myVGGish, self).__init__()
          self.fc2 = torch.nn.Linear(300, 128)
          self.relu2 = torch.nn.ReLU()
          self.fc3 = torch.nn.Linear(128, 2) 
    

    def forward(self, x):
          x = self.fc2(x)
          x = self.relu2(x)
          x = self.fc3(x)
          return x


model = myVGGish()

Device to use: 0


myVGGish(
  (fc2): Linear(in_features=300, out_features=128, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [16]:
tensorboard_writer = SummaryWriter('./tensorboard_logs')

In [17]:
def plot_signal(signal, title, cmap=None):
    fig = plt.figure()
    if signal.ndim == 1:
        plt.plot(signal)
    else:
        plt.imshow(signal, cmap=cmap)    
    plt.title(title)
    
    plot_buf = io.BytesIO()
    plt.savefig(plot_buf, format='jpeg')
    plot_buf.seek(0)
    plt.close(fig)
    return ToTensor()(PIL.Image.open(plot_buf))

In [32]:
def train(model, epoch, loader):
    model.train()
    for batch_idx, (sounds, sample_rate, inputs, labels) in enumerate(loader['train_loader']):
        # print(inputs.shape, torch.min(inputs), torch.max(inputs))
        # print(labels.shape, torch.min(labels), torch.max(labels))
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        # _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        iteration = epoch * len(loader['train_loader']) + batch_idx
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(inputs), len(loader['train_loader'].dataset), 
                          100. * batch_idx / len(loader['train_loader']), loss))
            tensorboard_writer.add_scalar('training loss/loss', loss, iteration)
            tensorboard_writer.add_scalar('learning rate/lr', optimizer.param_groups[0]['lr'], iteration)
                
        
        # if batch_idx % debug_interval == 0:    # report debug image every "debug_interval" mini-batches
        #     for n, (inp, pred, label) in enumerate(zip(inputs, predicted, labels)):
        #         series = 'label_{}_pred_{}'.format(classes[label.cpu()], classes[pred.cpu()])
        #         tensorboard_writer.add_image('Train MelSpectrogram samples/{}_{}_{}'.format(batch_idx, n, series), 
        #                                      plot_signal(inp.cpu().numpy().squeeze(), series, 'hot'), iteration)

In [41]:
import sklearn

def test(model, epoch, loader):
    model.eval()
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    true = []
    predict = []
    with torch.no_grad():
        for idx, (sounds, sample_rate, inputs, labels) in enumerate(loader['test_loader']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            _, predicted = torch.max(outputs, 1)
            true = true + labels.tolist()
            predict = predict + predicted.tolist()
            c = (predicted == labels)
            for i in range(len(inputs)):
                label = labels[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1
        
            iteration = (epoch + 1) * len(loader['test_loader'])

    total_accuracy = 100 * sum(class_correct)/sum(class_total)
    balanced_accuracy = sklearn.metrics.balanced_accuracy_score(true, predict)
    f_score = sklearn.metrics.f1_score(true, predict)
    precision = sklearn.metrics.precision_score(true, predict)
    recall = sklearn.metrics.recall_score(true, predict)
    cm = sklearn.metrics.confusion_matrix(true, predict)
    print('[Iteration {}] Accuracy on the {} test audios: {}%\n'.format(epoch, sum(class_total), total_accuracy))
    print('[Iteration {}] balanced accuracy on the {} test audios: {}%\n'.format(epoch, sum(class_total), balanced_accuracy))
    print('[Iteration {}] f_score on the {} test audios: {}%\n'.format(epoch, sum(class_total), f_score))
    print('[Iteration {}] precision on the {} test audios: {}%\n'.format(epoch, sum(class_total), precision))
    print('[Iteration {}] recall on the {} test audios: {}%\n'.format(epoch, sum(class_total), recall))
    print('[Iteration {}] cm on the {} test audios: {}%\n'.format(epoch, sum(class_total), cm))

### RESULTS

In [42]:

def deploy(loader):
    optimizer = optim.SGD(model.parameters(), lr = configuration_dict.get('base_lr', 0.001), momentum = 0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = configuration_dict.get('number_of_epochs')//3, gamma = 0.1)
    criterion = nn.CrossEntropyLoss()

    device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device('cpu')
    print('Device to use: {}'.format(device))
    model.to(device)

    log_interval = 10
    debug_interval = 25
    for epoch in range(configuration_dict.get('number_of_epochs', 10)):
        print(epoch)
        train(model, epoch, loader)
        test(model, epoch, loader)
        scheduler.step()

deploy(string['v'])

Device to use: 0
0
Train Epoch: 0 [0/884 (0%)]	Loss: 0.378158
Train Epoch: 0 [640/884 (71%)]	Loss: 0.362083
[Iteration 0] Accuracy on the 277.0 test audios: 84.47653429602889%

[Iteration 0] balanced accuracy on the 277.0 test audios: 0.5%

[Iteration 0] f_score on the 277.0 test audios: 0.9158512720156556%

[Iteration 0] precision on the 277.0 test audios: 0.8447653429602888%

[Iteration 0] recall on the 277.0 test audios: 1.0%

[Iteration 0] cm on the 277.0 test audios: [[  0  43]
 [  0 234]]%

1


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 1 [0/884 (0%)]	Loss: 0.378142
Train Epoch: 1 [640/884 (71%)]	Loss: 0.362079
[Iteration 1] Accuracy on the 277.0 test audios: 84.47653429602889%

[Iteration 1] balanced accuracy on the 277.0 test audios: 0.5%

[Iteration 1] f_score on the 277.0 test audios: 0.9158512720156556%

[Iteration 1] precision on the 277.0 test audios: 0.8447653429602888%

[Iteration 1] recall on the 277.0 test audios: 1.0%

[Iteration 1] cm on the 277.0 test audios: [[  0  43]
 [  0 234]]%

2
Train Epoch: 2 [0/884 (0%)]	Loss: 0.378127
Train Epoch: 2 [640/884 (71%)]	Loss: 0.362073
[Iteration 2] Accuracy on the 277.0 test audios: 84.47653429602889%

[Iteration 2] balanced accuracy on the 277.0 test audios: 0.5%

[Iteration 2] f_score on the 277.0 test audios: 0.9158512720156556%

[Iteration 2] precision on the 277.0 test audios: 0.8447653429602888%

[Iteration 2] recall on the 277.0 test audios: 1.0%

[Iteration 2] cm on the 277.0 test audios: [[  0  43]
 [  0 234]]%

3
Train Epoch: 3 [0/884 (0%)]	Lo

In [43]:
deploy(string['c'])

Device to use: 0
0
Train Epoch: 0 [0/464 (0%)]	Loss: 0.361673
[Iteration 0] Accuracy on the 146.0 test audios: 81.5068493150685%

[Iteration 0] balanced accuracy on the 146.0 test audios: 0.5%

[Iteration 0] f_score on the 146.0 test audios: 0.8981132075471698%

[Iteration 0] precision on the 146.0 test audios: 0.815068493150685%

[Iteration 0] recall on the 146.0 test audios: 1.0%

[Iteration 0] cm on the 146.0 test audios: [[  0  27]
 [  0 119]]%

1


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 1 [0/464 (0%)]	Loss: 0.361665
[Iteration 1] Accuracy on the 146.0 test audios: 81.5068493150685%

[Iteration 1] balanced accuracy on the 146.0 test audios: 0.5%

[Iteration 1] f_score on the 146.0 test audios: 0.8981132075471698%

[Iteration 1] precision on the 146.0 test audios: 0.815068493150685%

[Iteration 1] recall on the 146.0 test audios: 1.0%

[Iteration 1] cm on the 146.0 test audios: [[  0  27]
 [  0 119]]%

2
Train Epoch: 2 [0/464 (0%)]	Loss: 0.361658
[Iteration 2] Accuracy on the 146.0 test audios: 81.5068493150685%

[Iteration 2] balanced accuracy on the 146.0 test audios: 0.5%

[Iteration 2] f_score on the 146.0 test audios: 0.8981132075471698%

[Iteration 2] precision on the 146.0 test audios: 0.815068493150685%

[Iteration 2] recall on the 146.0 test audios: 1.0%

[Iteration 2] cm on the 146.0 test audios: [[  0  27]
 [  0 119]]%

3
Train Epoch: 3 [0/464 (0%)]	Loss: 0.361652
[Iteration 3] Accuracy on the 146.0 test audios: 81.5068493150685%

[Iteration 3] b

In [46]:
deploy(string['b'])

Device to use: 0
0
Train Epoch: 0 [0/101 (0%)]	Loss: 0.806964
[Iteration 0] Accuracy on the 32.0 test audios: 50.0%

[Iteration 0] balanced accuracy on the 32.0 test audios: 0.5%

[Iteration 0] f_score on the 32.0 test audios: 0.6666666666666666%

[Iteration 0] precision on the 32.0 test audios: 0.5%

[Iteration 0] recall on the 32.0 test audios: 1.0%

[Iteration 0] cm on the 32.0 test audios: [[ 0 16]
 [ 0 16]]%

1


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 1 [0/101 (0%)]	Loss: 0.806929
[Iteration 1] Accuracy on the 32.0 test audios: 50.0%

[Iteration 1] balanced accuracy on the 32.0 test audios: 0.5%

[Iteration 1] f_score on the 32.0 test audios: 0.6666666666666666%

[Iteration 1] precision on the 32.0 test audios: 0.5%

[Iteration 1] recall on the 32.0 test audios: 1.0%

[Iteration 1] cm on the 32.0 test audios: [[ 0 16]
 [ 0 16]]%

2
Train Epoch: 2 [0/101 (0%)]	Loss: 0.806868
[Iteration 2] Accuracy on the 32.0 test audios: 50.0%

[Iteration 2] balanced accuracy on the 32.0 test audios: 0.5%

[Iteration 2] f_score on the 32.0 test audios: 0.6666666666666666%

[Iteration 2] precision on the 32.0 test audios: 0.5%

[Iteration 2] recall on the 32.0 test audios: 1.0%

[Iteration 2] cm on the 32.0 test audios: [[ 0 16]
 [ 0 16]]%

3
Train Epoch: 3 [0/101 (0%)]	Loss: 0.806788
[Iteration 3] Accuracy on the 32.0 test audios: 50.0%

[Iteration 3] balanced accuracy on the 32.0 test audios: 0.5%

[Iteration 3] f_score on the 32.0 tes

In [47]:
deploy(string['all'])

Device to use: 0
0
Train Epoch: 0 [0/1451 (0%)]	Loss: 0.409194
Train Epoch: 0 [640/1451 (43%)]	Loss: 0.396095
Train Epoch: 0 [1280/1451 (87%)]	Loss: 0.459058
[Iteration 0] Accuracy on the 454.0 test audios: 81.05726872246696%

[Iteration 0] balanced accuracy on the 454.0 test audios: 0.5%

[Iteration 0] f_score on the 454.0 test audios: 0.8953771289537712%

[Iteration 0] precision on the 454.0 test audios: 0.8105726872246696%

[Iteration 0] recall on the 454.0 test audios: 1.0%

[Iteration 0] cm on the 454.0 test audios: [[  0  86]
 [  0 368]]%

1


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 1 [0/1451 (0%)]	Loss: 0.409130
Train Epoch: 1 [640/1451 (43%)]	Loss: 0.396079
Train Epoch: 1 [1280/1451 (87%)]	Loss: 0.459006
[Iteration 1] Accuracy on the 454.0 test audios: 81.05726872246696%

[Iteration 1] balanced accuracy on the 454.0 test audios: 0.5%

[Iteration 1] f_score on the 454.0 test audios: 0.8953771289537712%

[Iteration 1] precision on the 454.0 test audios: 0.8105726872246696%

[Iteration 1] recall on the 454.0 test audios: 1.0%

[Iteration 1] cm on the 454.0 test audios: [[  0  86]
 [  0 368]]%

2
Train Epoch: 2 [0/1451 (0%)]	Loss: 0.409099
Train Epoch: 2 [640/1451 (43%)]	Loss: 0.396061
Train Epoch: 2 [1280/1451 (87%)]	Loss: 0.458965
[Iteration 2] Accuracy on the 454.0 test audios: 81.05726872246696%

[Iteration 2] balanced accuracy on the 454.0 test audios: 0.5%

[Iteration 2] f_score on the 454.0 test audios: 0.8953771289537712%

[Iteration 2] precision on the 454.0 test audios: 0.8105726872246696%

[Iteration 2] recall on the 454.0 test audios: 1.0%

